In [8]:
import pandas as pd
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np

In [24]:
cannabis = pd.read_csv('..\data\cannabis.csv')
disease = pd.read_csv('..\data\Disease.csv')

In [25]:
print(cannabis.describe(), cannabis.head())
print('----------------------')
print(disease.describe(), disease.head())

            Rating
count  2351.000000
mean      4.310634
std       0.836394
min       0.000000
25%       4.200000
50%       4.400000
75%       4.700000
max       5.000000            Strain    Type  Rating                                     Effects  \
0          100-Og  hybrid     4.0  Creative,Energetic,Tingly,Euphoric,Relaxed   
1  98-White-Widow  hybrid     4.7    Relaxed,Aroused,Creative,Happy,Energetic   
2            1024  sativa     4.4   Uplifted,Happy,Relaxed,Energetic,Creative   
3        13-Dawgs  hybrid     4.2     Tingly,Creative,Hungry,Relaxed,Uplifted   
4        24K-Gold  hybrid     4.6   Happy,Relaxed,Euphoric,Uplifted,Talkative   

                      Flavor  \
0        Earthy,Sweet,Citrus   
1      Flowery,Violet,Diesel   
2    Spicy/Herbal,Sage,Woody   
3  Apricot,Citrus,Grapefruit   
4       Citrus,Earthy,Orange   

                                         Description  
0  $100 OG is a 50/50 hybrid strain that packs a ...  
1  The ‘98 Aloha White Widow is an espe

In [26]:
print(cannabis.isna().sum())
print(disease.isna().sum())

Strain          0
Type            0
Rating          0
Effects         0
Flavor         46
Description    33
dtype: int64
condition       0
daily_dosage    0
schedule        0
method          6
dtype: int64


In [29]:
disease = disease.replace(np.NaN, 'no specific recommendations')

In [34]:
cannabis[cannabis['Flavor'].isna() == True]

,Strain,Type,Rating,Effects,Flavor,Description
103,Amethyst,hybrid,3.9,"Energetic,Creative,Giggly,Hungry,Happy",NaN,Amethyst is a hybrid cannabis strain from the ...
123,Arabian-Gold,sativa,4.1,"Euphoric,Tingly,Giggly,Sleepy,Creative",NaN,Arabian Gold is a heavy sativa strain of myste...
160,Bad-Azz-Kush,hybrid,5.0,None,NaN,Bad Azz Kush by Barney’s Farm was created with...
215,Birds-Eye,sativa,5.0,"Happy,Uplifted,Energetic,Euphoric,Relaxed",NaN,Birds Eye is a variety of Jack Herer grown by ...
364,Boombaye,hybrid,0.0,None,NaN,Boombaye by Mr. Mack’s Snack is a CBD-rich str...
379,Broke-Diesel,hybrid,5.0,"Uplifted,Euphoric,Creative,Energetic,Happy",NaN,Broke Diesel is a 50/50 hybrid marijuana strai...
484,Chem-Jong-Ill,hybrid,5.0,"Uplifted,Creative,Happy,Talkative,Energetic",NaN,This sativa-dominant cross is for cannabis pro...
574,Confidential-Wreck,hybrid,4.2,"Hungry,Relaxed,Tingly,Uplifted,Creative",NaN,"Confidential Wreck, also known as Lohan, is th..."
673,Do-Over-Og,hybrid,4.7,"Talkative,Uplifted,Creative,Euphoric,Happy",NaN,Do-Over OG is an indica-dominant strain that c...
679,Domino,indica,4.0,"Sleepy,Happy,Tingly,Uplifted,Euphoric",NaN,NaN


In [35]:
cannabis['Description'].iloc[160]

'Bad Azz Kush by Barney’s Farm was created with the help of rapper Bad Azz of Dogg Pound Gangsta Crips. While the full story remains untold, Bad Azz contributed to this new Kush cut that brings traditional OG elements to the table while elevating yield and flavor. Bad Azz Kush is known to express hash and berry aromas with an earthy, spicy flavor on the palate. It is a cross of OG Kush, Purple Urkle, and Afghan Kush. This strain’s weighted physical effects and carefree euphoria make it a perfect end of the day smoke.\xa0'

In [ ]:
#Todo: Use LSTM model to add effects and flavor from description